In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import data_exploration_toolbox as ex # my own toolkit for data exploration
from matplotlib import pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
        
populations = pd.read_csv('/kaggle/input/history-of-demographics-and-wars/population.csv')
countries = pd.read_csv('/kaggle/input/countries-of-the-world/countries of the world.csv')
battles = pd.read_csv('/kaggle/input/history-of-demographics-and-wars/HCED Data v2.csv', encoding = 'latin-1')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'data_exploration_toolbox'

In [ ]:
print('Rows: ', populations.shape[0], 'Columns: ', populations.shape[1], '\n')
populations.sample(5)

In [ ]:
print('Rows: ', countries.shape[0], 'Columns: ', countries.shape[1], '\n')
countries.sample(5)

In [ ]:
print('Rows: ', battles.shape[0], 'Columns: ', battles.shape[1], '\n')
battles.sample(8)

In [ ]:
populations = populations.rename(columns = {'Entity': 'area', 'Code': 'code', 'Year': 'year', 'Population (historical estimates)': 'pop_estimate'})
populations.sample(5)

And then get a general overview:

In [ ]:
populations.describe()

In [ ]:
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['year']
  ys = series['pop_estimate']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = populations.sort_values('year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('Population (historical estimates)')

In [ ]:
populations[populations.area == 'World'].sort_values('year')

In [ ]:
sns.barplot(
    x = populations[populations.area == 'World'].year,
    y = populations[populations.area == 'World'].pop_estimate
)

plt.xticks(rotation=45)
plt.show()

In [ ]:
ex.col_types(populations)

In [ ]:
print('Number of countries: ', len(populations.area.unique()))
print('Number of country codes: ', len(populations.code.unique()))

In [ ]:
code_2_entity = ex.map_col2col(populations.area, populations.code)

code_2_entity

There. Let's see how many countries have exactely one code:

In [ ]:
one_code_entities = code_2_entity.index.unique()[code_2_entity.apply(sum, axis = 1) == 1]
many_codes_entities = code_2_entity.index.unique()[code_2_entity.apply(sum, axis = 1) > 1]
no_code_entities = code_2_entity.index.unique()[code_2_entity.apply(sum, axis = 1) == 0]

In [ ]:
print(
    'Number of entities: ', populations.area.unique().size, '\n\n',
    'Number of entities with exactely 1 code: ', len(one_code_entities), '\n\n',
    'Number of entities with more than 1 code: ', len(many_codes_entities), '\n\n',
    'Number of entities with no code: ', len(no_code_entities), '\n\n',
)

In [ ]:
no_code_entities

In [ ]:
# Selecting fields
countries = countries[['Country', 'Region', 'Area (sq. mi.)']]

# Renaming fields
countries = countries.rename(columns = {'Country': 'country', 'Region': 'region', 'Area (sq. mi.)': 'area_mi2'})

In [ ]:
countries.sample(5)

In [ ]:
ex.col_types(countries)

In [ ]:
countries.describe()

In [ ]:
countries.sort_values(by = 'area_mi2')

In [ ]:
countries = countries.rename(columns = {'area_mi2': 'area_km2'})
countries

Let's also check uniqueness:

In [ ]:
countries.country.value_counts()

In [ ]:
countries.region.value_counts()

In [ ]:
missing_countries = pd.DataFrame({
    'country': ['Montenegro', 'South Sudan'],
    'region': ['EASTERN EUROPE', 'SUB-SAHARAN AFRICA'],
    'area_km2': ['13812', '644329']
})

missing_countries

In [ ]:
countries = pd.concat([countries, missing_countries])
countries

In [ ]:
countries = countries.sort_values(by=['region', 'country'])
countries

In [ ]:
# Selecting cols
battles = battles[['ID', 'Country', 'Year', 'War', 'Lehmann Zhukov Scale', 'Infered Scale', 'Participants', 'Participant 1', 'Participant 2', 'Winner', 'Loser', 'Latitude', 'Longitude']]

# Renaming cols
battles = battles.rename(columns = {'ID': 'id', 'Country': 'country', 'Year': 'year', 'War': 'war', 'Lehmann Zhukov Scale': 'LZ_scale', 'Infered Scale': 'infered_scale', 'Participants': 'participants', 'Participant 1': 'participant_1', 'Participant 2': 'participant_2', 'Winner': 'winner', 'Loser': 'loser', 'Latitude': 'latitude', 'Longitude': 'longitude'})

In [ ]:
battles

In [ ]:
# Taking only lines from 'battles' where the line IS NOT all NaN:
battles = battles.loc[~battles.isna().all(axis=1)]
battles

In [ ]:
ex.col_types(battles)

In [ ]:
battles.id.value_counts().sort_values()

In [ ]:
battles[battles.id == 'Beijing1644']

In [ ]:
battles[battles.id == 'Hull1643']

In [ ]:
battles[battles.id == 'Hoyerswerda1813']

Let's disambiguiate:

In [ ]:
battles.loc[1929, 'id'] = battles.loc[1929, 'id'] + 'a'
battles.loc[1925, 'id'] = battles.loc[1925, 'id'] + 'b'
battles.loc[1925, 'war'] = 'unknown' # Believe me this is not a waste of code
battles.loc[7229, 'id'] = 'Hull1642'
battles.loc[7153, 'id'] = 'Hoyerswerda1759'

In [ ]:
battles.id.value_counts().sort_values()

There.

In [ ]:
pd.DataFrame({'year': [s for s in battles.year if len(str(s)) > 4]}).sample(10)

In [ ]:
def first_year(date):
    # This function either takes a single year 'yyyy' and returns it as integer,
    # or takes two years as 'yyyy-yyyy' and retrurns only one as integer.
    # Returns 'unknown' if the input is float (like a NaN)
    # If date is an integer already, it returns it as it is
    
    if isinstance(date, int): 
        return date # This runs in case date has already been formatted by ref_year
    
    if pd.isna(date): return date       # Leaving Null values
    if date[0] == '-': return int(date) # Returning '-476' as -476 (negative is BC)
    else: date = date.split('-') # Splitting '1983-1984' into ['1983', '1984']
        
    if len(date) == 1: return int(date[0]) # Returns 1983 if there was only '1983' to begin with
    if date[0] == '': return int(date[1])  # Returns 1983 if input was '1983-'
    if date[1] == '': return int(date[0])  # Returns 1984 if input was '-1984'
    
    return int(date[0]) # Returns 1983 if input was '1983-1984'

def last_year(date):
    # This function either takes a single year 'yyyy' and returns it as integer,
    # or takes two years as 'yyyy-yyyy' and retrurns only one as integer.
    # Returns 'unknown' if the input is float (like a NaN)
    # If date is an integer already, it returns it as it is
    
    if isinstance(date, int): 
        return date # This runs in case date has already been formatted by ref_year
    
    if pd.isna(date): return date       # Leaving Null values
    if date[0] == '-': return int(date) # Returning '-476' as -476 (negative is BC)
    else: date = date.split('-') # Splitting '1983-1984' into ['1983', '1984']
        
    if len(date) == 1: return int(date[0]) # Returns 1983 if there was only '1983' to begin with
    if date[0] == '': return int(date[1])  # Returns 1983 if input was '1983-'
    if date[1] == '': return int(date[0])  # Returns 1984 if input was '-1984'
    
    return int(date[1]) # Returns 1984 if input was '1983-1984'

In [ ]:
first_year = battles.year.map(first_year)
last_year = battles.year.map(last_year)

battles.loc[:,'duration'] = last_year - first_year + 1

battles.loc[:,'year'] = first_year

In [ ]:
battles

In [ ]:
print(
    ex.col_types(battles[['year', 'duration']]),
    '\n random sample: \n',
    battles[['year', 'duration']].sample(12)
)

In [ ]:
sns.histplot((battles[['year']][battles.year<2021]))

In [ ]:
battles[pd.isna(battles.year)]

In [ ]:
battles.loc[7423,['year']] = 1807
battles.loc[7423,['duration']] = 1
battles.loc[12707,['year']] = 1768
battles.loc[12707,['duration']] = 1

In [ ]:
battles.year.sort_values()

In [ ]:
battles.duration.sort_values()

In [ ]:
battles.duration.value_counts()

In [ ]:
battles[battles.duration == -1798]

In [ ]:
battles.loc[11315, 'duration'] = 1

In [ ]:
battles[battles.duration > 1].sample(12)

In [ ]:
ex.col_types(battles)

In [ ]:
battles.loc[:,'year'] = battles.year.map(int)
battles.loc[:,'duration'] = battles.duration.map(int)

In [ ]:
battles.participants

In [ ]:
ex.nan2nan(battles, 'participants', 'participant_1')

In [ ]:
ex.nan2nan(battles, 'participants', 'participant_2')

In [ ]:
ex.nan2nan(battles, 'participants', 'winner')

In [ ]:
ex.nan2nan(battles, 'participants', 'winner')

In [ ]:
ex.nan2nan(battles, 'winner', 'loser')

In [ ]:
battles = battles[['id', 'war', 'country', 'year', 'duration', 'LZ_scale', 'infered_scale', 'participants','winner', 'loser','latitude','longitude']]
battles

In [ ]:
# Defining a function that turns a string '['a', 'b', 'c']' into a list of strings:
def str2list(string):
    if not pd.isna(string):             # Replaces NaN with empty list
        string = string.strip('[]\' ')  # Removing trailing [' and ']
        string = string.split('\', \'') # Separating partecipants into list elements
    return string

battles.loc[:,'participants'] = battles.participants.map(str2list)

In [ ]:
battles[battles.latitude.isna()]

In [ ]:
# Roughly the coordinates of the center of Mongolia (47 N, 104 E):
battles.latitude[6095] = float(47)
battles.longitude[6095] = float(-104)

In [ ]:
battles[battles.participants.map(lambda x: x == [''])]

In [ ]:
for i in battles.index[battles.participants.isna()]:
    w = battles.loc[i, 'winner']
    l = battles.loc[i, 'loser']
    if not pd.isna(w) and not pd.isna(l):
        battles.at[i, 'participants'] = [w, l]
    else:
        battles.at[i, 'participants'] = 'unavailable_data'

In [ ]:
empty_list_indexes = battles.participants.map(lambda x: x == [''])

for i in battles.index[empty_list_indexes]:
    w = battles.loc[i, 'winner']
    l = battles.loc[i, 'loser']
    if not pd.isna(w) and not pd.isna(l):
        battles.at[i, 'participants'] = [w, l]
    else:
        battles.at[i, 'participants'] = 'unavailable_data'

In [ ]:
battles.sample(12)

In [ ]:
battles[battles.loser.isna()]

In [ ]:
battles[battles.loser.isna()][['winner']].value_counts()

In [2]:
accidental_incidents = battles[(battles.loser.isna()) & (battles.winner == 'Incident')].index
battles.loc[accidental_incidents, 'winner'] = 'Incident'

incident = battles[(battles.loser.isna()) & (battles.winner == 'Incident')].index
battles.loc[incident, 'loser'] = battles.loc[incident, 'loser'].fillna('Incident')

massacres = battles[(battles.loser.isna()) & (battles.winner == 'Massacre')].index
battles.loc[massacres, 'loser'] = battles.loc[massacres, 'loser'].fillna('Massacre')

draws = battles[(battles.loser.isna()) & (battles.winner == 'Draw')].index
battles.loc[draws, 'loser'] = battles.loc[draws, 'loser'].fillna('Draw')

battles[battles.loser.isna()][['winner']].value_counts()

NameError: name 'battles' is not defined

In [ ]:
battles[battles.loser.isna()]

In [3]:
battles.winner.fillna('unknown', inplace = True)
battles.loser.fillna('unknown', inplace = True)
battles[battles.loser.isna()]

NameError: name 'battles' is not defined

In [ ]:
ex.initials(battles.country),

In [ ]:
battles[battles.country.isna()]

In [ ]:
battles.loc[651,'country'] = 'Israel/Syria'

In [ ]:
battles.loc[:,'country'] = battles.country.map(lambda x: x.strip())

ex.initials(battles.country)

In [ ]:
ex.finals(battles.country)

Those are acceptable. Let's see who end with '.' :

In [ ]:
battles[battles.country.map(lambda s: s.endswith('.'))]

In [ ]:
print(
    ex.initials(battles.war),
    '\n',
    ex.finals(battles.war)
)


In [ ]:
battles[battles.war.isna()]

In [ ]:
battles.loc[:,'war'] = battles.war.fillna('unknown')

Let's now check those that begin or end with brackets:

In [ ]:
battles[battles.war.map(lambda x: x.startswith('['))]

In [ ]:
battles[battles.war.map(lambda x: x.endswith(']'))]

Looks like those square brakets are really there forno reason, so:

In [ ]:
battles.loc[:,'war'] = battles.war.map(lambda x: x.strip('[]\' '))

In [ ]:
battles.sample(12)

There.

# | 4.6 Size

Each battle has a size, an indicator of its magnitude. `LZ_scale` is the Lehmann-Zhukov scale for battle sizes, and `infered_scale` is still a mistery. Let's explore:

In [4]:
ex.nan2nan(battles, 'LZ_scale', 'infered_scale')

NameError: name 'ex' is not defined

In [ ]:
battles.LZ_scale.value_counts()

According to the data documnetation, that's the meaning of the Lehmann-Zhukov scale for battle sizes:

"Battle Scale:

1 - No more than 5,000 men on either side

2 - Between 5,000 and 20,000 men on either side

3 - Between 20,000 and 100,000 men on either side

4 - Between 100,000 and 500,000 men on either side

5 - Between 500,000 and 1,000,000 men on either side

6 - Over 1,000,000 men on either side"

While I'm not sure what kind of information `infered_scale` holds:

In [ ]:
battles.infered_scale.value_counts()

This column does not seem to be about any interesting numerical attribute and I feel confident dropping it:

In [ ]:
battles = battles.drop(columns = ['infered_scale'])

As for `LZ_scale`, let's see the types:

In [ ]:
ex.col_types(battles[['LZ_scale']])

A scale of 1-2-3-4-5-6 makes more sense as integer, while NULL values can just filled with -1:

In [ ]:
battles.LZ_scale = [x for x in battles.LZ_scale.map(lambda x: int(x) if pd.notna(x) else -1)]

Here's how they're distributed:

In [ ]:
sns.histplot(battles.LZ_scale[battles.LZ_scale != -1])

In [ ]:
ex.col_types(battles[['LZ_scale']])

# | 4.7 Coordinates
For some reason, longitude is set as string:

In [ ]:
ex.col_types(battles[['latitude', 'longitude']])

In [ ]:
battles.loc[:,'longitude'] = battles.longitude.map(float)

In [ ]:
ex.col_types(battles[['latitude', 'longitude']])

Done.

# 5. Uniforming Topography
Each of the three table has a coulmn where it indicates the names of countries or areas the data refers to. If the three tables have to work in conjunction, they must have the same names for the same places, and ideally these names will have 1 to 1 match across tables.

We might need those matchings to use this column a key for selection or merging. So the first thing that needs doing is conforming the three lists of countries.

In [ ]:
print(
    populations.area.unique().size,
    countries.country.unique().size,
    battles.country.unique().size
)

All three tables have about 200 countries, but, as we've seen before, `populations` includes in `area` names of countries, continents, and other identifiable vast areas (such as Middle-East, or South-Pacific). Let's check out the formats:

In [ ]:
# Renaming 'country' columns to 'area':
countries = countries.rename(columns = {'country': 'area'})
battles = battles.rename(columns = {'country': 'area'})

# Printing the first 10 areas:
print(
    populations.area.sort_values().unique()[0:10], '\n\n',
    countries.area.sort_values().unique()[0:10], '\n\n',
    battles.area.sort_values().unique()[0:10]
)

Before anything else, let's see if there is anything we'd want to strip:

In [ ]:
#
print(
    ex.initials(populations.area), '\n\n',
    ex.initials(countries.area), '\n\n',
    ex.initials(battles.area)
)

In [ ]:
print(
    ex.finals(populations.area), '\n\n',
    ex.finals(countries.area), '\n\n',
    ex.finals(battles.area)
)

Let's make a few changes:

In [ ]:
def conform(s):
    s = s.strip(' .')             # getting rid of trailing spaces
    s = s.lower()                 # conforming all to lower case
    s = s.replace(' ', '_')       # conforming all to '_' as separator
    s = s.replace('(', '')        # getting rid of '(' characters
    s = s.replace(')', '')        # getting rid of ')' characters
    s = s.replace('.', '')        # getting rid of '.' characters
    s = s.replace(',', '_')        # getting rid of '.' characters
    s = s.replace('&', 'and')     # getting rid of '&' in favor of 'and'
    s = s.replace('__', '_')
    return s

populations.area = populations.area.map(conform)
countries.area = countries.area.map(conform)
battles.area = battles.area.map(conform)

Now we shold be looking at something a little bit more reasonable to compare:

In [ ]:
print(
    populations.area.sort_values().unique()[0:10], '\n\n',
    countries.area.sort_values().unique()[0:10], '\n\n',
    battles.area.sort_values().unique()[0:10]
)

How close are we now to having matching columns? Consider that I'll write a wrangling notebook where I want to merge first `countries` to `populations`, as to add surface area information. That will be constant throughout the years. Then I'll merge `battles` to `population` as to add information about the battles that occurred in each country, year by year.

In order to do this, I'll call merges or joins on the columns `year` and `area` (country). While the year is pretty straight forward, I have to make sure that both `battles.area` and `countries.area` actually map onto `populations.area`. It's going to get hairy.

Let's start by checking whether any value in `countries.area` has no matching value to `populations.area`:

In [ ]:
# Do all values in countries.area have a match in populations.area?
ex.foreign_k2k(countries.area, 
           populations.area, 
           df1_name = 'countries', 
           df2_name = 'populations', 
           n_matches = 0,
           mode = '='
           )

In [ ]:
# Do all values in battles.area have a match in populations.area?
ex.foreign_k2k(battles.area, 
           populations.area,
           df1_name = 'battles',
           df2_name = 'populations',
           n_matches = 0,
           mode = '='
    )

Looks like there's a few records in both tables that would be lost if I left-joined them right now to `populations`. Many are due to different names for the same place, some to typos. Battle places are often not proper coutries. For example, Sint Maarten is anctually part of the Netherlands, and won't match anything in `populations.area`, which only accounts for actual countries.

The following code will map the area names in all three tables to a better common groud (I've checked the three area lists almost name by name).

In [5]:
# UPDATE list for countries.area
countries_area_names_update = {
    'bahamas_the': 'bahamas',
    'british_virgin_is': 'british_virgin_islands',
    'burma': 'myanmar',
    'central_african_rep': 'central_african_republic',
    'congo_dem_rep': 'democratic_republic_of_the_congo', # These are actually separate countries
    'congo_repub_of_the': 'republic_of_the_congo',
    'czech_republic': 'czech_republic',                  # This will be redefined in the populations table
    'cote_d\'ivoire': 'ivory_coast',
    'gambia_the': 'gambia',
    'gaza_strip': 'gaza_strip', 
    'korea_north': 'north_korea',
    'korea_south': 'south_korea',
    'macau': 'macao',
    'virgin_islands': 'usa_virgin_islands',
    'macedonia': 'macedonia',                            # This will be redefined in the other tables
    'micronesia_fed_st': 'micronesia',
    'n_mariana_islands': 'northern_mariana_islands',
    'st_pierre_and_miquelon': 'saint_pierre_and_miquelon',
    'swaziland': 'eswatini',                             # Seems unlisted in 'populations'
    'turks_and_caicos_is': 'turks_and_caicos_islands',
    'west_bank': 'west_bank',                            # This will be merged with gaza
}


# UPDATE list for battles.area
battles_area_names_update = {
    'sicily': 'italy',
    'swaziland': 'eswatini',                        # The country recently changed name
    'czechia': 'czech_republic',
    'britain': 'united_kingdom',
    'united_kingom': 'united_kingdom',
    'namhaedo_south_korea': 'south_korea',
    'bosnia_and_hercegovina': 'bosnia_and_herzegovina',
    'macedonia_fyrom': 'macedonia',
    'holland': 'netherlands',
    'phillipines': 'philippines',
    'south_africsa': 'south_africa',
    'congo': 'republic_of_the_congo',
    'falkland_islands_islas_malvinas': 'argentina', # Geographically, it's the closest country
    'the_netherlands': 'netherlands',
    'ivory_coast': 'ivory_coast',
    'korea': 'south_korea',                         # Checked geodetic coordinates on googlemaps
    'mariana_islands': 'northern_mariana_islands',
    'mauritannia': 'mauritania',
    'paardeberg_drift_south_africa': 'south_africa',
    'kora_samarahan_malaysia': 'malaysia',
    'kosovo': 'serbia',
    'moldavia': 'moldova',
    'ile_de_reunion_france': 'reunion',
    'ile_de_reunion': 'reunion',
    'virgin_islands_us': 'usa_virgin_islands',
    'isle_of_man_united_kingdom': 'isle_of_man',
    'brussels_belgium': 'belgium',
    'sint_maarten': 'netherlands',                   # saint_kitts_and_nevis is closest country, but it's actually part of Netherlands
    'st_kitts_and_nevis': 'saint_kitts_and_nevis',
    'st_lucia': 'saint_lucia',
    'st_vincent_and_the_grenadines': 'saint_vincent_and_the_grenadines',
    'northalerton_united_kingdom': 'united_kingdom',
    'micronesia_fed_sts': 'micronesia',
    'saudi_arabi': 'saudi_arabia',
    'south_african_republic': 'south_africa',
    
    # Unclear attribution:
    'israel/syria': 'syria',
    'palesinian_territories': 'palestine',
    'west_bank_and_gaza': 'palestine',
    'ukraine_belarus_poland': 'ukraine',# Geodetic coordinates place the battle in Ukraine
    
    # Sea battles
    'north_atlantic_ocean': 'brazil',   # Research confirms the two battles occurred in "north_atlantic_ocean" were actually close to Brasil, as coordinates indicate
    'atlantic_ocean': 'ireland',        # Ireland looks like the closest land on the map
    'north_sea': 'north_sea',           # 4 battles, 3 of which have the same coordinates next to uk. 4th one is in Norway
    'persian_gulf': 'iran',             # The closest country to these coordinates (1 occurrence only) is Iran
    
    # Old reference?
    'east_indies': 'indonesia',         # 1 reference only, right in the Javan Sea 
}


# UPDATE list for populations.area
populations_area_names_update = {
    'czechia': 'czech_republic',
    'democratic_republic_of_congo': 'democratic_republic_of_the_congo',
    'congo': 'republic_of_the_congo',
    'united_states_virgin_islands': 'usa_virgin_islands',
    'micronesia_country': 'micronesia',
    'cote_d\'ivoire': 'ivory_coast',
    'sint_maarten_dutch_part': 'sint_maarten',
    'north_macedonia': 'macedonia',     # 'battles' makes the distinction btw macedonia and macedonia fyrom
}

# UPDATING:
countries.area = countries.area.map(lambda x: countries_area_names_update[x] if x in countries_area_names_update else x)
battles.area = battles.area.map(lambda x: battles_area_names_update[x] if x in battles_area_names_update else x)
populations.area = populations.area.map(lambda x: populations_area_names_update[x] if x in populations_area_names_update else x)

# populations has entries in 'area' for 'serbia', 'montenegro', and 'serbia_and_montenegro'.
# serbia_and_montenegro will be considered superflous:

populations = populations[populations.area != 'serbia_and_montenegro']

NameError: name 'countries' is not defined

In [ ]:
# Checking improvements after update:
ex.foreign_k2k(countries.area, 
           populations.area, 
           df1_name = 'countries', 
           df2_name = 'populations', 
           n_matches = 0,
           mode = '='
    )

In [ ]:
# Checking improvements after update:
ex.foreign_k2k(battles.area, 
           populations.area, 
           df1_name = 'battles', 
           df2_name = 'populations', 
           n_matches = 0,
           mode = '='
    )

The situation has improved quite a lot, but there still are a couple of things to take care of. `populations.area` actually lists Palestine, so battles that took place in `gaza_strip` and `west_bank` can be mapped to Palestine. `countries` has to be updated with a record for Palestine and its surface area then:

In [ ]:
# Combining at gaza and west_bank

# Calculating palestinian surface area:
west_bank_km2 = countries.area_km2[countries.area == 'west_bank'][222]
gaza_strip_km2 = countries.area_km2[countries.area == 'gaza_strip'][74]
palestine_km2 = west_bank_km2 + gaza_strip_km2

# Defining countries record for Palestine:
palestine = pd.DataFrame({
    'area': ['palestine'],
    'region': ['NEAR EAST'],
    'area_km2': [palestine_km2]
    })

# Adding Palestine to countries:
countries = pd.concat([countries, palestine])

# Removing Gaza and West Bank from countries:
countries = countries[(countries.area != 'west_bank') & (countries.area != 'gaza_strip')]

countries

As for battles in the North Sea, let's take a look at what we have:

In [ ]:
battles[battles.area == 'north_sea']

The coordinates (54.997090 2.9813061) are pretty close to the UK, while (64.450002 6.464478) can be assigned to Norway. I'll use geographical information rather than looking at participants, because the field `area` lists where the battle took place rather than who was in it.

In [ ]:
battles.area[battles.id == 'Glowworm1940'] = 'norway'
battles.area[battles.area == 'north_sea'] = 'united_kingdom'

battles[battles.id.map(lambda x: 'Dogger Bank' in x)]

In [ ]:
battles[battles.id.map(lambda x: 'Glowworm1940' in x)]

And that is that.

# 6. Exporting Clean Tables
This is as far as I'm willing to gowith cleaning this data. I'd love to map `winner` and `loser` to `participants`, but that looks like a life long task destined to fail. Besides, I don-t think I'm going to make use of the `winner` and `loser` fields in the analysis. So if anybody wants to have more fun with this, here's the export:

In [ ]:
battles.to_csv('/kaggle/working/battles_clean.csv')
populations.to_csv('/kaggle/working/populations_clean.csv')
countries.to_csv('/kaggle/working/countries_areas.csv')

A word of caution: the `battles` fields `participants`, `winner`, and `loser` might require additional work depending on the use you're going to make of this data. Some `participants` list might be incomplete and need filling from `winner` and `loser`, or vice versa.

As for the geographical areas, many of those still listed in the table `populations` are not actual countries, and might not be match to anything in the other two tables.


Finally, notice that the populations listed in `participants`, `winner`, and `loser` won't map well onto `area`. Not because of any flaw in the dataset, but because the mapping of populations to locations throughout history would be very complicated.

Here's a final check of the three tables:

In [ ]:
battles.sample(12)

In [ ]:
populations.sample(12)

In [6]:
countries.sample(12)

NameError: name 'countries' is not defined